<a href="https://colab.research.google.com/github/amanjaiswal777/Fundamentals-Machine-Learning-algorithms/blob/master/Tensorflow/Tensorflow_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing the tensorflow package using pip
!pip install tensorflow

     |████████████████████████████████| 462 kB 4.3 MB/s 


In [2]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.8.0


In [3]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [4]:
# Loading and preparing the MNIST dataset

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


#Adding a channel dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [6]:
#Building Keras model

class MyModel(Model):
  def __init__(self):
      super(MyModel, self).__init__()
      self.conv1 = Conv2D(32, 3, activation='relu')
      self.flatten = Flatten()
      self.d1 = Dense(128, activation='relu')
      self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)


#Creating an instance of the Model
model = MyModel()

In [7]:
#Choosing an optimizer and loss fuction for training
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name='test_accuracy')

In [10]:
# Using tf.GradientTape to train the model

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  

In [11]:
# Testing the model

@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [15]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
      f'Epoch {epoch + 1},   '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result() * 100},  '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result() * 100},  '
  )

Epoch 1,   Loss: 0.005658371839672327, Accuracy: 81.09749603271484,  Test Loss: 0.06627660244703293, Test Accuracy: 78.4129409790039,  
Epoch 2,   Loss: 0.005163165275007486, Accuracy: 73.78356170654297,  Test Loss: 0.07108616083860397, Test Accuracy: 75.70453643798828,  
Epoch 3,   Loss: 0.004342608619481325, Accuracy: 72.68340301513672,  Test Loss: 0.07581076771020889, Test Accuracy: 75.34237670898438,  
Epoch 4,   Loss: 0.0026103626005351543, Accuracy: 66.13031768798828,  Test Loss: 0.09704621136188507, Test Accuracy: 66.93081665039062,  
Epoch 5,   Loss: 0.0034773785155266523, Accuracy: 60.061668395996094,  Test Loss: 0.06991179287433624, Test Accuracy: 61.73271942138672,  
